# <div class="alert alert-block alert-info" style="border-width:4px">How to load dependent libraries in user functions </div>

This notebook walks through on how to load dependent libraries in a user function.

Every user now has a private folder under his session, in the location /home/jovyan/src. This folder is only available for a specific user. All sessions under the user share this directory. 

Any python files under this directory is automatically available in the PYTHONPATH of the jupyter notebooks and cluster jobs. This will help the user organize the code better.

An example is given below.


Execute the below cell to make a file called "mylib.py" under /home/jovyan/src. 

In [ ]:
%%writefile /home/jovyan/src/mylib.py


## A simple test function
def my_pretty_function():
    print("*"*100)
    print("pretty function")
    print("!"*100)


## A model builder function which will get called from user model function
def my_special_mnist_modelfn(features, labels, mode, params):
    import tensorflow as tf

    net = tf.feature_column.input_layer(features, [tf.feature_column.numeric_column("data", shape=(784))])
    # labels = tf.one_hot(labels, 2) ## either or
    for units in [20, 20]:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)

    # Compute logits (1 per class).
    # logits = tf.layers.dense(net, 2, activation=None) ## either or
    logits = tf.layers.dense(net, 10, activation=None)

    # Compute predictions.
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Compute loss.
    loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)

    # Compute evaluation metrics.
    labels_to_compare = tf.argmax(labels, 1)
    accuracy = tf.metrics.accuracy(labels=labels_to_compare,
                                   predictions=predicted_classes,
                                   name='acc_op')
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)

    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN

    global_step = tf.train.get_global_step()
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(loss, global_step=global_step)

    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op,
                                      training_chief_hooks=None
                                      )

Now below we start a regular job which uses this dependent module. Note that the model function loads the function from the dependent module.

In [ ]:
from sbrain.learning.experiment import *
from sbrain.dataset.dataset import *
import time
import os


def input_function(mode, batch_size, params):
    from tensorflow.examples.tutorials.mnist import input_data
    import tensorflow as tf
    from mylib import my_pretty_function

    my_pretty_function()

    local_dir = "/workspace/shared-dir/sample-notebooks/demo-data/learning/mnist/"

    if mode == "train":
        mnist = input_data.read_data_sets(local_dir, one_hot=True)

        dataset = tf.data.Dataset.from_tensor_slices(({"data" : mnist.train.images}, mnist.train.labels))
        dataset = dataset.shuffle(1000).batch(batch_size).repeat()
        return dataset
    else:
        mnist = input_data.read_data_sets(local_dir, one_hot=True)

        dataset = tf.data.Dataset.from_tensor_slices(({"data" : mnist.test.images}, mnist.test.labels))
        dataset = dataset.batch(batch_size)
        return dataset




def my_model_function_with_mnist(features, labels, mode, params):

    from mylib import my_pretty_function, my_special_mnist_modelfn

    my_pretty_function()

    return my_special_mnist_modelfn(features, labels, mode, params)

We submit the job.

In [ ]:
estimator = Estimator.NewClassificationEstimator(model_fn=my_model_function_with_mnist)
name = "BestModelEstimator" + str(time.time()).replace(".", "")
estimator = Estimator.create(name, "Hello", estimator)

hyper_parameters = HParams(iterations=5000, batch_size=10)
rc = RunConfig(no_of_ps=1, no_of_workers=1, summary_save_frequency=5000, run_eval=True, use_gpu=False, checkpoint_frequency_in_steps=500)

exper = Experiment.run(experiment_name="BestModelEstimator" + str(time.time()).replace(".", ""),
                       description="Really first model",
                       estimator=estimator,
                       hyper_parameters=hyper_parameters,
                       run_config=rc,
                       dataset_version_split=None,
                       input_function=input_function)
job = exper.get_single_job()
print(job.__dict__)
print("")
print("tensorboard url")
print(job.get_tensorboard_url())

job.has_finished()

job.wait_until_finish()

print("Is the job success?? : {}".format(job.is_success()))

print()
print("Model metrics..")
print(job.get_model().model_metrics)

This shows how to use dependent modules under /home/jovyan/src folder in the user functions.

CAVEAT: Please note that this is a dynamically loaded library. Even if the estimator is saved with a previous version of dependent files, when the job is run, it will point to the latest version of dependent module.